In [79]:
!pip install -q pyomo

In the following example, we will solve a slightly general optimization problem: 
\begin{align}
\min \ c_1 x_1 + c_2 x_2 + \ldots + c_N x_N, \\
{\rm{s.t.}} \;  x_1 + x_2 + \ldots +  x_N & \leq b_1, \\
w_1 x_1 + w_2 x_2 + \ldots + w_N x_N  & =b_2, \\
x_i & \leq u_i, \quad i=1,\ldots,N, \\
x_i & \geq l_i, \quad i=1,\ldots,N. \\
\end{align}
---

In [80]:
from pyomo.environ import *

In [81]:
import numpy as np

In [82]:
#creating a concrete model
model=ConcreteModel()

In [83]:
#defining no. of variables
N=10

In [84]:
#defining coefficient
obj_coef_c=np.array([-10.1, 20.15, 60.5, 80.05, 0.15, 40.5, -0.05, -30.02, 40.65, 0.125])

In [85]:
#defining constraints coefficient
const2_coef_w=np.array([0, 1.6, -2.02, 0, 1.01, -2.005, 0, 1.39, 1, -1.214])

In [86]:
#defining RHS value in constraint equation
const1_rhs_b1 = 151.2
const2_rhs_b2 = 44.6

In [87]:
#defining lower and upper bound for the variables
lower_bound = np.array([-np.inf,1,0,0,7,0,-np.inf,-np.inf,1,1])
upper_bound = np.array([4, 3,np.inf, 2, 10,np.inf, 13, 20,np.inf, 21])

In [88]:
col_indices = np.arange(N)
print(col_indices)

[0 1 2 3 4 5 6 7 8 9]


In [89]:
#declare the decision variables in the model
model.x = Var(col_indices)

In [90]:
# creating  a ConstraintList to hold multiple constraints
model.constraints = ConstraintList()

In [91]:
#defining constraint  x1+x2+…+xN<=b1
model.constraints.add(sum(model.x[j] for j in col_indices)<=const1_rhs_b1)

In [92]:
#defining constraint  w1x1+w2x2+…+wNxN=b2
model.constraints.add(sum(const2_coef_w[j]*model.x[j] for j in col_indices) == const2_rhs_b2)

In [93]:
#adding the upper bound and lower bounds for the variables
for j in col_indices:
  model.x[j].setlb(lower_bound[j])
  model.x[j].setub(upper_bound[j])

In [94]:
#defining objective function
model.objective=Objective(expr=sum(obj_coef_c[j]*model.x[j] for j in col_indices),sense=minimize)

In [95]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {1, 2}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

1 Var Declarations
    x : Size=10, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :  -inf :  None :   4.0 : False :  True :  Reals
          1 :   1.0 :  None :   3.0 : False :  True :  Reals
          2 :   0.0 :  None :   inf : False :  True :  Reals
          3 :   0.0 :  None :   2.0 : False :  True :  Reals
          4 :   7.0 :  None :  10.0 : False :  True :  Reals
          5 :   0.0 :  None :   inf : False :  True :  Reals
          6 :  -inf :  None :  13.0 : False :  True :  Reals
          7 :  -inf :  None :  20.0 : False :  True :  Reals
          8 :   1.0 :  None :   inf : False :  True :  Reals


In [96]:
!apt-get install -y -qq glpk-utils

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 144556 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Setting up libcolamd2:amd64 (1:5.1.2-2) ...
Setting up libamd2:amd64 

In [97]:
SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: -452.7909
  Upper bound: -452.7909
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 11
  Number of nonzeros: 18
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.013328313827514648
# ----------------------------------------------------------
#   Solution Information
# --------------------------

In [98]:
# display solution
print('\nObjective = ', model.objective())

print('\nDecision Variables')
for k in col_indices:
  print('x[',k,'] = ', model.x[k].value)

print('\nConstraints')
model.constraints.display()


Objective =  -452.7909

Decision Variables
x[ 0 ] =  4.0
x[ 1 ] =  3.0
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  10.0
x[ 5 ] =  0.0
x[ 6 ] =  13.0
x[ 7 ] =  20.0
x[ 8 ] =  3.114
x[ 9 ] =  1.0

Constraints
constraints : Size=2
    Key : Lower : Body               : Upper
      1 :  None :             54.114 : 151.2
      2 :  44.6 : 44.599999999999994 :  44.6
